# Root Cause Analysis

An iterative process in ITOps used to explore cause-and-effect relationships of an underlying problem. IT

In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf

## Data Preprocessing

In [4]:
#Load the dataset
data=pd.read_csv("root_cause_analysis.csv")
data.head()

,ID,CPU_LOAD,MEMORY_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003,ROOT_CAUSE
0,1,0,0,0,0,1,0,1,MEMORY
1,2,0,0,0,0,0,0,1,MEMORY
2,3,0,1,1,0,0,1,1,MEMORY
3,4,0,1,0,1,1,0,1,MEMORY
4,5,1,1,0,1,0,1,0,NETWORK_DELAY


In [5]:
print(data.dtypes)

ID              int64
CPU_LOAD        int64
MEMORY_LOAD     int64
DELAY           int64
ERROR_1000      int64
ERROR_1001      int64
ERROR_1002      int64
ERROR_1003      int64
ROOT_CAUSE     object
dtype: object


In [6]:
#Convert root cause data to a numeric value

from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
data['ROOT_CAUSE'] = label_encoder.fit_transform(data['ROOT_CAUSE'])

#Convert Pandas DataFrame to a numpy vector
np_data = data.to_numpy().astype(float)

#Extract the feature variables (X)
X_train = np_data[:,1:8]

#Extract the target variable (Y), convert to one-hot-encoding
Y_train=np_data[:,8]
Y_train = tf.keras.utils.to_categorical(Y_train,3)

print("Shape of feature variables :", X_train.shape)
print("Shape of target variable :",Y_train.shape)

Shape of feature variables : (1000, 7)
Shape of target variable : (1000, 3)


## Building model

In [7]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2

#Training Parameters
EPOCHS=20
BATCH_SIZE=100
VERBOSE=1
OUTPUT_CLASSES=len(label_encoder.classes_)
N_HIDDEN=128
VALIDATION_SPLIT=0.2

#Create a Keras sequential model
model = tf.keras.models.Sequential()
#Add a Dense Layer
model.add(keras.layers.Dense(N_HIDDEN,input_shape=(7,),name='Dense-Layer-1',activation='relu'))

#Add a second dense layer
model.add(keras.layers.Dense(N_HIDDEN,name='Dense-Layer-2',activation='relu'))

#Add a softmax layer for categorial prediction
model.add(keras.layers.Dense(OUTPUT_CLASSES,name='Final',activation='softmax'))

#Compile the model, using Adam optimizer
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

#Build the model
model.fit(X_train,Y_train,batch_size=BATCH_SIZE,epochs=EPOCHS,verbose=VERBOSE,validation_split=VALIDATION_SPLIT)
model.summary()

Epoch 1/20
8/8 [==============================] - 6s 47ms/step - loss: 1.0434 - accuracy: 0.5475 - val_loss: 0.9424 - val_accuracy: 0.7700
Epoch 2/20
8/8 [==============================] - 0s 7ms/step - loss: 0.8688 - accuracy: 0.8213 - val_loss: 0.7913 - val_accuracy: 0.8000
Epoch 3/20
8/8 [==============================] - 0s 7ms/step - loss: 0.7153 - accuracy: 0.8150 - val_loss: 0.6631 - val_accuracy: 0.8000
Epoch 4/20
8/8 [==============================] - 0s 8ms/step - loss: 0.5872 - accuracy: 0.8175 - val_loss: 0.5681 - val_accuracy: 0.7800
Epoch 5/20
8/8 [==============================] - 0s 7ms/step - loss: 0.5037 - accuracy: 0.8313 - val_loss: 0.5277 - val_accuracy: 0.8000
Epoch 6/20
8/8 [==============================] - 0s 6ms/step - loss: 0.4566 - accuracy: 0.8400 - val_loss: 0.5149 - val_accuracy: 0.7900
Epoch 7/20
8/8 [==============================] - 0s 5ms/step - loss: 0.4382 - accuracy: 0.8438 - val_loss: 0.5156 - val_accuracy: 0.8000
Epoch 8/20
8/8 [=================

## Predicting

In [9]:
#Pass individual flags to Predict the root cause
CPU_LOAD=1
MEMORY_LOAD=0
DELAY=0
ERROR_1000=0
ERROR_1001=1
ERROR_1002=1
ERROR_1003=0

#prediction=model.predict_classes([[CPU_LOAD,MEMORY_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003]])
#print(label_encoder.inverse_transform(prediction))

predict_x=model.predict([[CPU_LOAD,MEMORY_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003]]) 
classes_x=np.argmax(predict_x,axis=1)

1/1 [==============================] - 3s 3s/step


In [10]:
print(label_encoder.inverse_transform(classes_x))

['DATABASE_ISSUE']
